# Install the following required libraries


In [ ]:
! pip install requests
! pip install pandas
! pip install matplotlib
! pip install geopandas
! pip install folium

# Import modules and libraries


In [1]:
import requests 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium

See below the **base_url**.  Note that this isn't an actual API link. It contains symbolic links.

To obtain the desired real url, a link based on the input keyword as search_term has to be selected. 

Especially the last part of this link in needed.

![plot](../images/symbolic_links_openbydata.png)

In [9]:
# the following link contains symbolic links which include keywords
base_url = 'https://opendata.bayern/api/hub/repo/datasets/' 

# Make an HTTP GET request to the provided base_url.
response = requests.get(base_url)
# will raise en exception if the request failed.
response.raise_for_status()
# parse the HTTP response content as JSON and return it.
data = response.json()

What happens next:
Add the last part of the sustained link to the base_url

By appending this last part to the base_url, a valid URL from which one can download the relevant dataset is generated.

In [19]:
# The example keyword to search for is
search_term = 'baumkataster' # in english 'tree register'

# Create a list of links that contain the specified search_term from the data.
symbolic_links = [link for link in data if search_term in link]
# If there are any symbolic_links found:
if symbolic_links:
    # Take the first link from symbolic_links.
    first_link = symbolic_links[0]
    # Extract the rear part of the link after the last '/'.
    last_part = first_link.split('/')[-1]
    # Assemble the full URL by appending the last part to the base_url.
    real_url = base_url + last_part
print(f'We created this link using the base_url and the last part of the link. \nreal_url : {real_url}')


The link is created by using the base_url and the last part of the link. 
real_url : https://opendata.bayern/api/hub/repo/datasets/baumkataster_stadt_wuerzburg-wuerzburg


what happens next

In [16]:
response_new = requests.get(real_url)
# will raise en exception if the request failed.
response_new.raise_for_status()
# Parse the HTTP response content as JSON 
data_new = response_new.json()
# Attempt to retrieve the '@graph' key from 'data_new', default to an empty list if not found.
graph = data_new.get('@graph', [])
# If the '@graph' key exists in the data:
if graph:
    # Extract the 'dcat:accessURL' from the first entry of 'graph', default to an empty dictionary if not found.
    access_url = graph[0].get('dcat:accessURL', {})
    # From the 'access_url', retrieve the '@id' key which should contain the desired URL.
    result_id = access_url.get('@id', None)
    # Check if the string 'geojson' is present in the 'result_id' (the desired URL).
    if 'geojson' in result_id:
        # If found, return this 'geojson' URL.
        geojson_url = result_id
    else:
        print("'geojson' not found in the resulting link")
else:
    print("No @graph key in the data")


print(geojson_url)

https://opendata.wuerzburg.de/api/v2/catalog/datasets/baumkataster_stadt_wuerzburg/exports/geojson


Let's plot with geopandas

In [ ]:
# Read the geojson data from the provided URL using GeoPandas and store it in 'gdf'.
gdf = gpd.read_file(geojson_url)
# Define coordinates for the city of Würzburg in Germany.
wuerzburg_coords = [49.79245, 9.932966]
# Create a folium map centered on Würzburg with a zoom level of 16.
m = folium.Map(location=wuerzburg_coords, zoom_start=16)
# Iterate over each row in the GeoDataFrame 'gdf'.
for _, row in gdf.iterrows():
    # Check if the current row has a valid geometry that is not empty.
    if row['geometry'] and not row['geometry'].is_empty:
        # Extract longitude and latitude from the geometry of the current row.
        lon, lat = row['geometry'].coords.xy
        # Create a circle marker on the folium map for each valid geometry.
        folium.CircleMarker(
            location=[lat[0], lon[0]],  # Set the position for the circle marker.
            radius=1,                   # Define the radius of the circle marker.
            color="green",              # Set the color of the circle marker.
            fill=True,                  # Fill the inside of the circle marker.
            fill_color="green"          # Set the fill color for the inside of the circle marker.
        ).add_to(m)                     # Add the created circle marker to the folium map 'm'.

m

Let's see the results

![plot](../images/plotting_of_dataset1.png)

![plot](../images/plotting_of_dataset2.png)

![plot](../images/plotting_of_dataset3.png)